In [1]:
%pip install -qU pypdf
%pip install -U langchain
%pip install -U langchain-community
%pip install -U langchain-groq
%pip install langchain-huggingface
%pip install langgraph
%pip install python-dotenv

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: /home/bender/.virtualenvs/flash-skills-rag-agentes-ia/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: /home/bender/.virtualenvs/flash-skills-rag-agentes-ia/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: /home/bender/.virtualenvs/flash-skills-rag-agentes-ia/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
/bin/bash: warning: setlocale: LC_ALL: cann

# Perguntando sobre Marte

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()  # Carrega variáveis do .env

api_key = os.getenv('GROQ_API_KEY')
huggingface_token = os.getenv('HUGGINGFACE_TOKEN')
print(api_key)  # Verifique se foi carregado corretamente
print(huggingface_token)

None
None


In [3]:
from langchain_groq import ChatGroq

In [4]:
llm = ChatGroq(model='llama-3.3-70b-versatile')

GroqError: The api_key client option must be set either by passing api_key to the client or by setting the GROQ_API_KEY environment variable

In [ ]:
prompt = """
    messages: Segundo a NASA quais seriam os benefícios científicos de ir para Marte?
    """

In [ ]:
resposta = llm.invoke(prompt)

In [ ]:
resposta

# Carregando um pdf

In [ ]:
url = 'https://raw.githubusercontent.com/allanspadini/curso-flash-rag/main/m2m_strategy_and_objectives_development.pdf'

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(url)
pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [ ]:
print(f"{pages[0].metadata}\n")
print(pages[0].page_content)

# Base de dados vetorial

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_huggingface import HuggingFaceEmbeddings

In [ ]:
import os
from huggingface_hub import InferenceClient

client = InferenceClient(
    provider="hf-inference",
    api_key=huggingface_token,
)
result = client.feature_extraction(
    "Today is a sunny day and I will get some ice cream.",
    model="mixedbread-ai/mxbai-embed-large-v1",
)



In [ ]:
%pip install sentence-transformers
embed_model = HuggingFaceEmbeddings(model_name="mixedbread-ai/mxbai-embed-large-v1")

In [ ]:
vector_store = InMemoryVectorStore.from_documents(pages, embed_model)

# Buscando informações

In [ ]:
docs = vector_store.similarity_search("Objectives Development Process", k=2)

In [ ]:
for doc in docs:
    print(f'Page {doc.metadata["page"]}: {doc.page_content[:300]}\n')

# Pipeline RAG

In [ ]:
retriever = vector_store.as_retriever()

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
template = """You're a helpful assistant that only gives answers bases on the given context. If the answer is not in the context, say "I don't know".

Context:
{context}

Question:
{question}

Answer:"""

In [ ]:
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
from IPython.display import display, Markdown

In [ ]:
response = chain.invoke("What are the Objectives Development Process?")
Markdown(response)

# Criando uma ferramenta

In [ ]:
from langchain_core.tools import tool

In [ ]:
@tool
def pega_contexto(query: str) -> str:
    """Pega o contexto baseado em uma pesquisa."""
    retriever = vector_store.as_retriever()
    resultado = retriever.invoke(query)
    return resultado

In [ ]:
async def carrega_pdf(url: str):
    loader = PyPDFLoader(url)
    pages = []
    async for page in loader.alazy_load():
        pages.append(page)
    vectorstore = InMemoryVectorStore.from_documents(pages, embed_model)
    return vectorstore

In [ ]:
import asyncio

async def load_vector_stores():
    global vector_store_agriculture, vector_store_dengue
    vector_store_agriculture = await carrega_pdf('https://raw.githubusercontent.com/allanspadini/curso-flash-rag/main/agriculture.pdf')
    vector_store_dengue = await carrega_pdf('https://raw.githubusercontent.com/allanspadini/curso-flash-rag/main/dengue.pdf')

await load_vector_stores()

In [ ]:
@tool
def pega_contexto_agriculture(query: str) -> str:
    """Pega o contexto sobre agricultura baseado em uma pesquisa."""
    retriever = vector_store_agriculture.as_retriever()
    resultado = retriever.invoke(query)
    return resultado

In [ ]:
@tool
def pega_contexto_dengue(query: str) -> str:
    """Pega o contexto sobre dengue baseado em uma pesquisa."""
    retriever = vector_store_dengue.as_retriever()
    resultado = retriever.invoke(query)
    return resultado

In [ ]:
tools = [pega_contexto,pega_contexto_agriculture,pega_contexto_dengue]

In [ ]:
pega_contexto_dengue.invoke("Cases of dengue we had since the beginning of 2025?")

# Agentic RAG

In [ ]:
from langgraph.prebuilt import create_react_agent

In [ ]:
system_prompt = """ You're a helpful assistant that only gives answers bases on the given context. If the answer is not in the context, say "I don't know"
    - pega_contexto: Tool that returns the context based on the users query if the query is about NASA and space travels.
    - pega_contexto_agriculture: Tool that returns the context based on the users query if the query is about agriculture.
    - pega_contexto_dengue: Tool that returns the context based on the users query if the query is about dengue.
"""

In [ ]:
agente_pdf = create_react_agent(model=llm, tools=tools, prompt=system_prompt)

In [ ]:
resultado= agente_pdf.invoke({"messages": [("user", "What are the Objectives Development Process?")]})

In [ ]:
resultado

In [ ]:
resultado= agente_pdf.invoke({"messages": [("user", "What causes dengue?")]})
resultado

In [ ]:
resultado= agente_pdf.invoke({"messages": [("user", "Why is agriculture crucial for India's economy, and what's its current need?")]})
resultado

# Definindo um grafo com memória

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import MessagesState
from langgraph.graph import START, StateGraph, END
from langgraph.prebuilt import tools_condition

from langgraph.prebuilt import ToolNode
from IPython.display import Image, display
from langchain_core.messages import HumanMessage, SystemMessage

In [ ]:
grafo = StateGraph(MessagesState)

In [ ]:
grafo.add_node("assistente", agente_pdf)
grafo.add_node("tools", ToolNode(tools))

In [ ]:
grafo.add_edge(START, "assistente")
grafo.add_conditional_edges("assistente",tools_condition,)

In [ ]:
grafo.add_edge("tools", "assistente")
grafo.add_edge("assistente",END)

In [ ]:
memoria = MemorySaver()
app = grafo.compile(checkpointer=memoria)

In [ ]:
Image(app.get_graph().draw_mermaid_png())

In [ ]:
def chat_com_memoria(mensagem_usuario: str, thread_id="1", verbose = False):
  config = {"configurable": {"thread_id": thread_id}}
  messages = app.invoke({"messages": [HumanMessage(content=mensagem_usuario)]}, config)
  if verbose:
        for message in messages['messages']:
            message.pretty_print()
  else:
      messages['messages'][-1].pretty_print()





In [ ]:
chat_com_memoria(mensagem_usuario="Why is agriculture crucial for India's economy, and what's its current need?", thread_id="2", verbose = False)

In [ ]:
chat_com_memoria(mensagem_usuario="What is the planet NASA is going?", thread_id="2", verbose = False)

In [ ]:
chat_com_memoria(mensagem_usuario="What planet again?", thread_id="2", verbose = False)